In [1]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

In [2]:
def plot_digit(image):
    """
    This function receives an image and plots the digit. 
    """
    plt.imshow(image, cmap='gray')
    plt.show()

In [3]:
def cnn(train_x, train_y, test1_x, test1_y):#, test2_x, test2_y):
    """
    Train and evaluate a feedforward network with two hidden layers.
    """
    # Add a single "channels" dimension at the end
    trn_x = train_x.reshape([-1, 28, 28, 1])
    tst1_x = test1_x.reshape([-1, 28, 28, 1])
    #tst2_x = test2_x.reshape([-1, 28, 28, 1])

    # First layer will need argument `input_shape=(30,30,1)`
    model = Sequential([
        # TODO: add your implementation here
        Conv2D(32,(5,5)),
        MaxPooling2D((2,2),(2,2)),
        Conv2D(64,(5,5)),
        MaxPooling2D((2,2),(1,1)),
        Flatten(input_shape=(30,30,1)),
        Dense(512,activation='relu'),
        Dense(10,activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(trn_x, train_y, epochs=10)

    print("Evaluating CNN on test set 1")
    model.evaluate(tst1_x, test1_y)
#     print("Evaluating CNN on test set 2")
#     return model.evaluate(tst2_x, test2_y)

In [4]:
def decenter(X,pad=2):
    #shifts values in X 2 pixels to the bottom right corner
    out = np.zeros([X.shape[0], X.shape[1]+abs(pad), X.shape[2]+abs(pad)])
    if pad > 0:
        out[:,pad:X.shape[1]+pad, pad:X.shape[2]+pad] = X
    else:
        out[:,0:X.shape[1], 0:X.shape[2]] = X
        
    return out

In [5]:
def h_theta(X, W, B):
    """
    For a given matrix W and vector B, this function predicts the value
    of each digit for each image of X. Here we assume that each column of X
    is a flattened image. 
    """
    return W.dot(X) + B 

In [6]:
def l5GradDesc(images, labels, images_test, test_labels):
    images = images.reshape(images.shape[0],28*28)
    images = images.T
    
    images_test = images_test.reshape(images_test.shape[0], 28*28)
    images_test = images_test.T
    
    # Learning rate alpha, for controlling the step of gradient descent
    alpha = 0.01

    # Number of instances in the training set
    m = images.shape[1]

    # Matrix W initialized with zeros
    W = np.zeros((10, 28*28))

    # Matrix B also initialized with zeros
    B = np.zeros((10,1))

    # Creating Y matrix where each column is an one-hot vector
    Y = np.zeros((10, m))
    for index, value in enumerate(labels):
        Y[value][index] = 1

    # Performs 1000 iterations of gradient descent
    # print("start W shape is ", W.shape)
    # print("start B shape is ", B.shape)
    for i in range(1000):
        # Write here your implementation of gradient descent
        ###tinashe start 
        temp = ((W.dot(images) + B - Y)).dot(np.transpose(images))
        w_pderivativ = (1/m) * temp
        b_pderivativ = (1/m) * (W.dot(images) + B - Y)
        b_pderivative = []
        for row in b_pderivativ:
            n_row= sum(row)
            b_pderivative.append(n_row)
        b_pderivative = np.array(b_pderivative).reshape(10,1)

        W = W - (alpha * w_pderivativ)
        B = B - (alpha * b_pderivative)
        ###tinashe end
        
    #make predictions for first test set
    Y_hat = W.dot(images_test) + B

    #one hot encode the results
    results = []
    for row in np.transpose(Y_hat):
        results.append(np.argmax(row))

    #get accuracy of the data based on the training results
    accurate = 0
    for i in range(len(results)):
        if results[i] == test_labels[i]:
            accurate += 1
    print("percentage accuracy is: ", accurate/len(results))

In [7]:
def main():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    #change x_train and x_test data to float from uint8
    x_train = x_train/255.0
    x_test = x_test/255.0
    
    #print(x_train.shape)
    #use one hot encoding to represent the answers ie labels for the data
    #both for training and testing
    with tf.compat.v1.Session():
        y_tran = tf.one_hot(y_train, 10).eval()
        y_tst = tf.one_hot(y_test, 10).eval()
    
    #shift the test data by 2 pixels to the bottom right
    x_test2 = decenter(x_test, 2)
    
    #call classification algorithms    
    #cnn(x_train, y_tran, x_test, y_tst) #################
    l5GradDesc(x_train, y_train, x_test, y_test)
    

In [8]:
main()

(60000, 28, 28)


KeyboardInterrupt: 